### Projection onto the symmetric subspace

In [ ]:
import json
import numpy as np
import pennylane as qml

from ansatz import Ansatz
from twirler.generators import get_ansatz_generators
from twirler.induced_representation import derive_unitaries_angle_embedding_analytic
from twirler.symmetry_groups import create_induced_subgroup_from_permutations, create_symmetric_group
from twirler.twirling import apply_twirling_to_generators

results = {}
n_qubits = 3

S = create_symmetric_group(n_qubits)
with open(f"groups/modified_subgroup_generators_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

subgroup_unitaries = {}
for k, groups in subgroups.items():
    subgroup_unitaries[k] = []
    for generators in groups:
        new_generators = [tuple(g) for g in generators]
        K = create_induced_subgroup_from_permutations(S, new_generators)
        unitaries = derive_unitaries_angle_embedding_analytic(K)
        subgroup_unitaries[k].append({"unitaries": unitaries, "subgroup": K})

results[n_qubits] = {}

for depth in range(1, 6):
    print(" Depth:", depth)
    results[n_qubits][depth] = {}
    for ansatz_id in range(1, 20):
        print("  Ansatz ID:", ansatz_id)
        super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
        ansatz = super_ansatz.get_ansatz()
        ansatz_generators = get_ansatz_generators(ansatz)

        ansatz_results = {}

        for k in subgroup_unitaries:
            # Precompute full generator matrices once per k
            G_full_list = []
            for gen_observable, wires, gate_name, theta in ansatz_generators:
                op = qml.Hermitian(gen_observable, wires=wires)
                G_full_list.append(qml.matrix(op, wire_order=range(n_qubits)))

            # Root-mean-square Frobenius distance across generators and subgroups
            total = 0.0
            count = 0

            for elem in subgroup_unitaries[k]:
                unitaries = elem["unitaries"]
                subgroup = elem["subgroup"]
                twirled_generators = apply_twirling_to_generators(
                    unitaries, ansatz_generators, n_qubits
                )

                for gen_idx in range(len(ansatz_generators)):
                    G_twirled = twirled_generators[gen_idx]["averaged"]
                    diff = G_full_list[gen_idx] - G_twirled
                    total += np.linalg.norm(diff, ord="fro")
                    count += 1

            avg_norm = total / count
            ansatz_results[k] = avg_norm

        results[n_qubits][depth][ansatz_id] = ansatz_results

with open(f"results/results_projection_onto_symmetric_subspace_{n_qubits}.json", "w") as f:
    json.dump(results, f, indent=4)